In [26]:
load_checkpoint= 0
experiment_path = 'yyy'

bool(load_checkpoint == experiment_path)

False

In [ ]:
import lightning as L
from pathlib import Path
from lightning.pytorch.callbacks import RichProgressBar
from lightning.pytorch.callbacks.progress.rich_progress import RichProgressBarTheme

from multimodal_bridge_matching import MultiModalBridgeMatching
from data.particle_clouds.jets import JetDataclass

from utils.dataloader import DataloaderModule
from utils.callbacks import JetsGenerativeCallback


######################################################################################
proj_dir = "/home/df630/Multimodal-Bridges/experiments/results/comet/multimodal-jets"
exp_dir = "5066786266c44cc893312f0531d6ba93"
ckpt = "last.ckpt"
######################################################################################

bridge = MultiModalBridgeMatching.load_from_checkpoint(Path(proj_dir) / exp_dir / "checkpoints" / ckpt)
config = bridge.config
config.experiment.dir_path = Path(proj_dir) / exp_dir

In [ ]:
progress_bar = RichProgressBar(theme=RichProgressBarTheme(**config.experiment.progress_bar.to_dict()))
generative_callback = JetsGenerativeCallback(config.clone())

In [ ]:
config.data.target.path = "/home/df630/CMB/data/AOJ/2016H_job9.h5"
config.data.target.delete("preprocess")
config.data.target.params.num_jets = 5000
config.train.batch_size = 1024
config.model.pipeline.num_timesteps = 100
config.train.data_split_frac = [0.0, 0.0, 1.0]

In [ ]:
# ...prep test data

jets = JetDataclass(config=config)
jets.preprocess()
dataloader = DataloaderModule(config=config, dataclass=jets)

# ...lightning model

generator = L.Trainer(accelerator="gpu", devices=[0], callbacks=[progress_bar, generative_callback],logger=False,)

# ...generate data

sample = generator.predict(bridge, dataloaders=dataloader.test)